In [7]:
%pip install pandas
from pathlib import Path
import pandas as pd

DATA_DIR = Path("data/raw")

PD_KW = dict(dtype_backend="pyarrow") 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/86/41/585a168330ff063014880a80d744219dbf1dd7a1c706e75ab3425a987384/pandas-2.3.3-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for numpy>=1.26.0 from https://files.pythonhosted.org/packages/32/ee/de999f2625b80d043d6d2d628c07d0d5555a677a3cf78fdf868d409b8766/numpy-2.3.3-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     -------------------- ------------------- 30.7/60.9 kB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 60.9/60.9 kB 1.6 MB/s eta 0:00:00
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/81/c4/34e93fe5f5429d7570ec1fa436f1986fb1f00c3e0f43a589fe2bbcd22c3f/pytz-2025.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/5c/23/c7abc0ca0a1526a0774eca151daeb8de62ec457

In [ ]:
def load_csv(path: Path, usecols=None, dtypes=None, nrows=None):
    """
    Carga un CSV a DataFrame de forma robusta.
    - usecols: columnas a leer (si quieres probar con un subconjunto).
    - dtypes: mapeo de tipos por columna si ya los conoces.
    - nrows: para muestrear primeras N filas si necesitas una carga rápida.
    """
    if not path.exists():
        raise FileNotFoundError(f"No existe: {path}")

    try:
        df = pd.read_csv(
            path,
            usecols=usecols,
            dtype=dtypes,
            low_memory=False,    # evita inferencias parciales
            on_bad_lines="warn", # avisa si hay líneas corruptas
            **PD_KW               # usa backend pyarrow si está disponible
        )
    except TypeError:
        df = pd.read_csv(
            path,
            usecols=usecols,
            dtype=dtypes,
            low_memory=False,
            on_bad_lines="warn",
        )
    return df


In [ ]:
path_chess   = DATA_DIR / "chessData.csv"
path_random  = DATA_DIR / "random_evals.csv"
path_tactic  = DATA_DIR / "tactic_evals.csv"

df_chess  = load_csv(path_chess)
df_random = load_csv(path_random)
df_tactic = load_csv(path_tactic)

In [ ]:
def overview(df: pd.DataFrame, name: str = "DataFrame", show_cols=20):
    print("=" * 100)
    print(f"📦 {name}")
    print("- Dimensiones:", df.shape)                 
    print("- Columnas:", len(df.columns))
    print("- Primeras columnas:", list(df.columns[:show_cols]))
    print("\n🔹 Tipos de datos:")
    display(df.dtypes.to_frame("dtype").head(show_cols))

    print("\n🔹 Memoria estimada por columna (Top 10):")
    mem = df.memory_usage(deep=True).sort_values(ascending=False).head(10)
    display((mem / (1024**2)).round(3).rename("MB").to_frame())

    print("\n🔹 df.info():")
    df.info(memory_usage="deep")

    print("\n🔹 df.head():")
    display(df.head(3))

    print("\n🔹 Valores únicos por columna (muestra):")
    display(df.nunique(dropna=True).sort_values(ascending=False).head(10).to_frame("nunique"))

    # (complemento mínimo para Paso 4, solo conteo bruto de nulos)
    print("\n🔹 Conteo de nulos (Top 10):")
    display(df.isna().sum().sort_values(ascending=False).head(10).to_frame("nulos"))
